<a href="https://colab.research.google.com/github/RohithOfRivia/Analysing-Conusmer-Behaviour-MRP/blob/main/Best_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install rdkit
!pip install -U mlxtend
!pip install Boruta

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from boruta import BorutaPy

import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import BorderlineSMOTE

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import MACCSkeys

from sklearn.cluster import KMeans

from sklearn.svm import OneClassSVM
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.ensemble import (GradientBoostingClassifier, 
                              HistGradientBoostingClassifier, IsolationForest)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import resample

from sklearn.metrics import f1_score, recall_score

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from xgboost import XGBClassifier
import time
import os
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 33.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.5 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 KB 4.5 MB/s eta 0:00:00


# Preprocessing and pipeline

In [ ]:
train_data_url = "https://raw.githubusercontent.com/RohithOfRivia/SMILES-Toxicity-Prediction/main/Data/train_II.csv"
test_data_url = "https://raw.githubusercontent.com/RohithOfRivia/SMILES-Toxicity-Prediction/main/Data/test_II.csv"

df = pd.read_csv(train_data_url)

In [ ]:
#transforming each compound into their canonical SMILES format. Optional.
def canonicalSmiles(smile):
    try:
        return Chem.MolToSmiles(Chem.MolFromSmiles(smile))
    except:
        return(Chem.MolToSmiles(Chem.MolFromSmiles("[Na+].[Na+].F[Si--](F)(F)(F)(F)F")))

In [ ]:
#Reads data and split up the given features
class FileReadTransform(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    #training and test data are slightly different, hence passing optional test param
    def transform(self, X, test=False):
        
        try:
          # if test == False:
            X['SMILES'] = X['Id'].apply(lambda x: x.split(';')[0])
            X['assay'] = X['Id'].apply(lambda x: x.split(';')[1])
        
        except KeyError:
            X['SMILES'] = X['x'].apply(lambda x: x.split(';')[0])
            X['assay'] = X['x'].apply(lambda x: x.split(';')[1])
          
        print("FileReadTransform done")
        
        #correct smiles for this compound found through https://www.molport.com/shop/index
        #X["SMILES"] = X["SMILES"].replace({"F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]":"[Na+].[Na+].F[Si--](F)(F)(F)(F)F"})
        
        #Deleting invalid compound from the data
        X = X.loc[X.SMILES != "F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]"]
        return X
    
#Converts each SMILES value to its respective canonical SMILES 
class CanonicalGenerator(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['SMILES'] = X['SMILES'].apply(canonicalSmiles)
        print("CanonicalGenerator done")
        return X


#Generate fingerprints for all compounds
class FingerprintGenerator(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
          #tracks each unique compound and its fingerprints
          tracker = []
          fps = []
          assays = []
          unique = len(X['SMILES'].unique())
          counter = 0

          for index, columns in X[["SMILES", "assay"]].iterrows():

              #skip if already in tracker
              if columns[0] in tracker:
                  continue

              #append each unique compound and theer respective fingerprints
              else:
                  tracker.append(columns[0])
                  assays.append(columns[1])

                  mol = Chem.MolFromSmiles(columns[0])
                  fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=256)
                  fps.append(fp.ToList())

                  counter += 1

                  # print(f"compound {counter}/{unique}...

          #Combining all compounds, assays and fingerprints into one dataframe 
          cols = a = ["x" + str(i) for i in range (1, 257)]
          smiles_df = pd.DataFrame(columns=['SMILES'], data=tracker)
          fingerprints = pd.DataFrame(columns=cols, data=fps)

          df = pd.concat([smiles_df, fingerprints], axis=1)

          print("FingerprintGenerator done")
          return pd.merge(X, df, on='SMILES') 

#Fingerprint generation for MACCS Keys
class FingerprintGeneratorM(BaseEstimator, TransformerMixin):
    

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
      
          #tracks each unique compound and its fingerprints
          tracker = []
          fps = []
          assays = []
          unique = len(X['SMILES'].unique())
          counter = 0

          for index, columns in X[["SMILES", "assay"]].iterrows():

              #skip if already in tracker
              if columns[0] in tracker:
                  continue

              #append each unique compound and thier respective fingerprints
              else:
                  

                  tracker.append(columns[0])
                  assays.append(columns[1])

                  mol = Chem.MolFromSmiles(columns[0])
                  fp = MACCSkeys.GenMACCSKeys(mol)
                  fps.append(fp.ToList())

                  counter += 1

                  # print(f"compound {counter}/{unique}...

          #Combining all compounds, assays and fingerprints into one dataframe 
          cols = a = ["x" + str(i) for i in range (1, 168)]
          smiles_df = pd.DataFrame(columns=['SMILES'], data=tracker)
          fingerprints = pd.DataFrame(columns=cols, data=fps)

          df = pd.concat([smiles_df, fingerprints], axis=1)

          print("FingerprintGenerator done")
          return pd.merge(X, df, on='SMILES') 


#Feature reduction with variance threshold 
class VarianceThresh(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, thresh=.8):
      
      #Looks to columns to determine whether X is training or testing data 
      cols = X.columns
      if 'x' in cols:
        temp_df = X.drop(columns=["x", "assay", "SMILES"])
        cols = ["x", "assay", "SMILES"]
      else:
        temp_df = X.drop(columns=["Id", "Expected","assay", "SMILES"])
        cols = ["Id", "Expected","assay", "SMILES"]

      #Selecting features based on the variance threshold
      selector = VarianceThreshold(threshold=(thresh * (1 - thresh))) 
      selector.fit(temp_df)

      #This line transforms the data while keeping the column names 
      temp_df = temp_df.loc[:, selector.get_support()]

      #Attaching the ids, assays, smiles etc. that is still required for model
      return pd.concat([X[cols], temp_df], axis=1) , selector


#Scale descriptors (Not used in this notebook)
class Scaler(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
      return self
    
    def transform(self, X):
      scaler = StandardScaler()

      if 'Id' in X.columns:
        temp_df = X.drop(columns=["Id", "Expected", "assay", "SMILES"])
        cols = ["Id", "Expected","assay", "SMILES"]

        X_scaled = pd.DataFrame(scaler.fit_transform(temp_df), columns=temp_df.columns)
        X = pd.concat([X[cols].reset_index(drop=True), X_scaled], axis=1)
          
        return X

      else:
        temp_df = X.drop(columns=["x", "assay", "SMILES"])
        cols = ["x", "assay", "SMILES"]

        X_scaled = pd.DataFrame(scaler.fit_transform(temp_df), columns=temp_df.columns)
        X = pd.concat([X[cols].reset_index(drop=True), X_scaled], axis=1)

        return X




# Generating descriptors

In [ ]:
class DescriptorGenerator(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
        return self
    
  def transform(self, X):
    #Initializing descriptor calculator
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    #Tracking each unique compound and generating descriptors 
    tracker = []
    descriptors = []
    for compound in X['SMILES']:

      if compound in tracker:
        continue

      else:
        tracker.append(compound)

        mol = Chem.MolFromSmiles(compound)
        current_descriptors = calc.CalcDescriptors(mol)
        descriptors.append(current_descriptors)

    # Combining X, SMILES, and generated descriptors 
    df = pd.DataFrame(descriptors,columns=desc_names)
    temp_df = pd.DataFrame(tracker, columns=["SMILES"])
    df = pd.concat([df, temp_df], axis=1)

    print("DescriptorGenerator done")
    return pd.merge(X, df, on='SMILES')

# Create Pipeline

In [ ]:
feature_generation_pipeline = Pipeline(steps=[
    ('read', FileReadTransform()),
     ('canon', CanonicalGenerator()),
     ('fpr', FingerprintGenerator()),
     ('desc', DescriptorGenerator())
     ])

df_processed = feature_generation_pipeline.fit_transform(df)
test_processed = feature_generation_pipeline.fit_transform(pd.read_csv(test_data_url))

FileReadTransform done
CanonicalGenerator done
FingerprintGenerator done
DescriptorGenerator done
FileReadTransform done
CanonicalGenerator done
FingerprintGenerator done
DescriptorGenerator done


# Feature Selection

In [ ]:
#Isolating the chemical descriptors for feature selection
descriptors = pd.concat([df_processed[['Id','SMILES','assay', 'Expected']], df_processed[df_processed.columns[-208:]]], axis=1)

In [ ]:
#Removing all rows 
descriptors.isna().sum().sum()

32552

In [ ]:
#Removing all columns which have NAN values
descriptors2 = descriptors.drop(columns=['BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW',
                      'BCUT2D_MRHI', 'BCUT2D_MRLOW'])
descriptors2 = descriptors2.dropna()

X = descriptors2.drop(['Id','SMILES','Expected'], axis=1)


y = descriptors2[['Expected']]

In [ ]:
#Removing all columns which have NAN values
X['assay'] = X['assay'].astype("int64")

#Features selected from BorutaPy
boruta_features2 = ['HeavyAtomMolWt','MaxPartialCharge','MinAbsPartialCharge','BertzCT','Chi1','Chi1n','Chi2v','Chi3n','Chi3v','Chi4v','LabuteASA','PEOE_VSA3','SMR_VSA6','SMR_VSA7','EState_VSA8','VSA_EState2','VSA_EState4','HeavyAtomCount','NumAromaticCarbocycles','MolLogP','MolMR','fr_Ar_OH','fr_COO','fr_COO2','fr_C_O','fr_C_O_noCOO','fr_amide','fr_benzene','fr_phenol','fr_phenol_noOrthoHbond','fr_sulfonamd','fr_thiazole','fr_thiophene','fr_urea','assay']

#Splitting data for training and validation. Mapping expected values from 2 to 1 because XGBoost does not support it for binary classification
X_train, X_test, y_train, y_test = train_test_split(X[boruta_features2], y['Expected'].map({2:0, 1:1}), test_size=0.2, random_state=0, stratify=y['Expected'].map({2:0, 1:1}))

In [ ]:
''' Optional to run this. Gives a list of features that pass the test as the output. 
The variable declaredboruta_features2 is a saved version of the output when the code was run for the best submission.
Output may vary slightly if executed again'''

# model = XGBClassifier(max_depth=6, learning_rate=0.01, n_estimators=600, colsample_bytree=0.3)

# boruta_features = []

# # let's initialize Boruta
# feat_selector = BorutaPy(
#     verbose=2,
#     estimator=model,
#     n_estimators='auto',
#     max_iter=10  # number of iterations to perform
# )

# # train Boruta
# # N.B.: X and y must be numpy arrays
# feat_selector.fit(np.array(X_train), np.array(y_train))

# # print support and ranking for each feature
# print("\n------Support and Ranking for each feature------")
# for i in range(len(feat_selector.support_)):
#     if feat_selector.support_[i]:
#         boruta_features.append(X_train.columns[i])
#         print("Passes the test: ", X_train.columns[i],
#               " - Ranking: ", feat_selector.ranking_[i])
#     else:
#         print("Doesn't pass the test: ",
#               X_train.columns[i], " - Ranking: ", feat_selector.ranking_[i])



' Optional to run this. Gives a list of features that pass the test as the output. \nThe variable declaredboruta_features2 is a saved version of the output when the code was run for the best submission.\nOutput may vary slightly if executed again'

In [ ]:
#This method trains the model with the training data and then prints the f1 score by using predictions from the holdout data
def train_model(xtrain, xtest, ytrain, ytest):
  model = XGBClassifier(seed = 20, max_depth=10, n_estimators=700 )
  model.fit(xtrain, ytrain)

  predictions = model.predict(xtest)
  print(f"F1 Score of model: {f1_score(predictions, ytest)}")

In [ ]:
train_model(X_train, X_test, y_train, y_test)

F1 Score of model: 0.6520947176684883


In [ ]:
#Now training with the entire dataset to make predictions on the test set

descriptors2['assay'] = descriptors2['assay'].astype('int64') 
model = XGBClassifier(seed = 20, max_depth=10, n_estimators=700 )
model.fit(descriptors2[boruta_features2], y['Expected'].map({2:0, 1:1}))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=700, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
#Changing assayID to int
test_processed['assay'] = test_processed['assay'].astype('int64')

#Making predicitons with the model
test_preds = model.predict(test_processed[boruta_features2])
test_preds

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
#Checking predictions for posititve and negative valeus
np.unique(test_preds, return_counts=True)

(array([0, 1]), array([9674, 1320]))

In [ ]:
#Converting the predictions into a dataframe
res = pd.DataFrame({})
res['Id'] = test_processed['x']
res['Predicted'] = test_preds

#Mapping expected values back to 2 and 1 
res['Predicted'] = res['Predicted'].map({0:2, 1:1})
res

,Id,Predicted
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,2
1,CC1=CC(=C(C=C1)C(C)(C)C)O;2451,2
2,CC1=CC(=C(C=C1)C(C)(C)C)O;2442,1
3,CC1=CC(=C(C=C1)C(C)(C)C)O;32,2
4,CC1=CC(=C(C=C1)C(C)(C)C)O;1382,2
...,...,...
10989,CC(=CC(=O)C)C;1856,2
10990,CCCCCCCCCC[N+](C)(C)CC1=CC=CC=C1.[Cl-];1848,1
10991,CC1=C(C(=O)N(N1C)C2=CC=CC=C2)N(C)CS(=O)(=O)[O-...,2
10992,COC1=CC=CC(=C1)C=O;2,2


In [ ]:
#Downloading the csv 
from google.colab import files

res.to_csv('28-03-23-2.csv', encoding = 'utf-8-sig', index=False) 
files.download('28-03-23-2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
res

,Id,Predicted2
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,2
1,CC1=CC(=C(C=C1)C(C)(C)C)O;2451,2
2,CC1=CC(=C(C=C1)C(C)(C)C)O;2442,1
3,CC1=CC(=C(C=C1)C(C)(C)C)O;32,2
4,CC1=CC(=C(C=C1)C(C)(C)C)O;1382,2
...,...,...
10989,CC(=CC(=O)C)C;1856,2
10990,CCCCCCCCCC[N+](C)(C)CC1=CC=CC=C1.[Cl-];1848,1
10991,CC1=C(C(=O)N(N1C)C2=CC=CC=C2)N(C)CS(=O)(=O)[O-...,2
10992,COC1=CC=CC(=C1)C=O;2,2
